In [23]:
# 1. ADVANCED CRUD OPERATIONS

from pymongo import MongoClient, InsertOne, UpdateOne, DeleteOne

# Koneksi ke client MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['university_db']
courses_collection = db['courses']

# Supaya data tidak double saat dijalankan ulang
courses_collection.delete_many({}) 

# Insert banyak data mata kuliah beserta jumlah mahasiswa terdaftar
operations = [
    InsertOne({'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematics'}),
    InsertOne({'course': 'CS 102', 'enrollments': 25, 'department': 'Computer Science'}),
    InsertOne({'course': 'History 201', 'enrollments': 20, 'department': 'History'}),
    InsertOne({'course': 'Physics 202', 'enrollments': 15, 'department': 'Physics'})
]
courses_collection.bulk_write(operations)
print('>> Data mata kuliah berhasil dimasukkan <<\n')
print('📋 DATA AWAL :\n')

# Menampilkan semua data di koleksi courses
for course in courses_collection.find():
    print(f"{course['course']} | {course['department']} | {course['enrollments']} mahasiswa")


# Kumpulan operasi dalam satu perintah bulk_write
ops = [
    # Tambah dua data baru (Insert)
    InsertOne({"course": "Biology 109", "enrollments": 18, "department": "Biology"}),
    InsertOne({"course": "Chemistry 105", "enrollments": 28, "department": "Chemistry"}),

    # Update satu data
    UpdateOne({"course": "Math 101"}, {"$set": {"enrollments": 15}}),

    # Hapus satu data
    DeleteOne({"course": "Physics 202"})
]

# Jalankan semua operasi sekaligus
result = courses_collection.bulk_write(ops)

print("\n>> Operasi bulk_write selesai <<\n")
print(f"Inserted: {result.inserted_count}, Updated: {result.modified_count}, Deleted: {result.deleted_count}\n")

# Menampilkan semua data terbaru
print('📋 DATA TERBARU :\n')
for course in courses_collection.find():
    print(f"{course['course']} | {course['department']} | {course['enrollments']} mahasiswa")


>> Data mata kuliah berhasil dimasukkan <<

📋 DATA AWAL :

Math 101 | Mathematics | 30 mahasiswa
CS 102 | Computer Science | 25 mahasiswa
History 201 | History | 20 mahasiswa
Physics 202 | Physics | 15 mahasiswa

>> Operasi bulk_write selesai <<

Inserted: 2, Updated: 1, Deleted: 1

📋 DATA TERBARU :

Math 101 | Mathematics | 15 mahasiswa
CS 102 | Computer Science | 25 mahasiswa
History 201 | History | 20 mahasiswa
Biology 109 | Biology | 18 mahasiswa
Chemistry 105 | Chemistry | 28 mahasiswa


In [25]:
# 2. COMPLEX FILTERING AND QUERYING

# Menampilkan semua data 
print('📋 DATA :\n')
for course in courses_collection.find():
    print(f"{course['course']} | {course['department']} | {course['enrollments']} mahasiswa")

# Mata kuliah dengan jumlah mahasiswa > 20
print("\n>> Mata kuliah dengan jumlah enrollments lebih dari 20 <<\n")
for course in courses_collection.find({'enrollments': {'$gt': 20}}):
    print(f"{course['course']} | {course['department']} | {course['enrollments']} mahasiswa")

# Menampilkan mata kuliah dari departemen tertentu
print("\n>> Mata kuliah dari departemen 'Computer Science' atau 'Mathematics' <<\n")
for course in courses_collection.find({'department': {'$in': ['Computer Science', 'Mathematics']}}):
    print(f"{course['course']} | {course['department']} | {course['enrollments']} mahasiswa")

📋 DATA :

Math 101 | Mathematics | 15 mahasiswa
CS 102 | Computer Science | 25 mahasiswa
History 201 | History | 20 mahasiswa
Biology 109 | Biology | 18 mahasiswa
Chemistry 105 | Chemistry | 28 mahasiswa

>> Mata kuliah dengan jumlah enrollments lebih dari 20 <<

CS 102 | Computer Science | 25 mahasiswa
Chemistry 105 | Chemistry | 28 mahasiswa

>> Mata kuliah dari departemen 'Computer Science' atau 'Mathematics' <<

Math 101 | Mathematics | 15 mahasiswa
CS 102 | Computer Science | 25 mahasiswa


In [26]:
# 3. AGGREGATION FRAMEWORK FOR DATA ANALYSIS

# Menampilkan semua data 
print('📋 DATA :\n')
for course in courses_collection.find():
    print(f"{course['course']} | {course['department']} | {course['enrollments']} mahasiswa")

# Menghitung rata-rata jumlah mahasiswa per departemen
pipeline = [
    {'$group': {'_id': '$department', 'average_enrollment': {'$avg': '$enrollments'}}}
]
print("\n>> Rata-rata jumlah mahasiswa per departemen <<\n")
for result in courses_collection.aggregate(pipeline):
    print(f"{result['_id']} | {result['average_enrollment']} mahasiswa")

# Mencari jumlah mahasiswa maksimum per departemen 
pipeline = [
    {'$group': {'_id': '$department', 'max_enrollment': {'$max': '$enrollments'}}}
]
print("\n>> Jumlah mahasiswa terbanyak per departemen <<\n")
for result in courses_collection.aggregate(pipeline):
    print(f"{result['_id']} | {int(result['max_enrollment'])} mahasiswa")



📋 DATA :

Math 101 | Mathematics | 15 mahasiswa
CS 102 | Computer Science | 25 mahasiswa
History 201 | History | 20 mahasiswa
Biology 109 | Biology | 18 mahasiswa
Chemistry 105 | Chemistry | 28 mahasiswa

>> Rata-rata jumlah mahasiswa per departemen <<

Computer Science | 25.0 mahasiswa
History | 20.0 mahasiswa
Mathematics | 15.0 mahasiswa
Biology | 18.0 mahasiswa
Chemistry | 28.0 mahasiswa

>> Jumlah mahasiswa terbanyak per departemen <<

Computer Science | 25 mahasiswa
History | 20 mahasiswa
Mathematics | 15 mahasiswa
Biology | 18 mahasiswa
Chemistry | 28 mahasiswa


In [27]:
# 4. DATA TRANSFORMATION USING PROJECT AND ADD FIELDS

# Menampilkan semua data 
print('📋 DATA :\n')
for course in courses_collection.find():
    print(f"{course['course']} | {course['department']} | {course['enrollments']} mahasiswa")

# Mengubah nama field dan menampilkan field tertentu dengan $project
pipeline = [
    {'$project': {'course_name': '$course','department_name': '$department','enrollments': 1}}
]
print("\n>> Hasil Projection <<\n")
for result in courses_collection.aggregate(pipeline):
    print(f"{result['course_name']} | {result['department_name']} | {result['enrollments']} mahasiswa")

# Menambahkan field baru berdasarkan kondisi dengan $addfields
pipeline = [
    {'$addFields': {'enrollment_category': {'$cond': {'if': {'$gt': ['$enrollments', 20]},'then': 'high','else': 'low'}}}}      
]
print("\n>> Hasil setelah menambah field baru <<\n")
for result in courses_collection.aggregate(pipeline):
    print(f"{result['course']} | {result['department']} | {result['enrollments']} mahasiswa | kategori: {result['enrollment_category']}")

📋 DATA :

Math 101 | Mathematics | 15 mahasiswa
CS 102 | Computer Science | 25 mahasiswa
History 201 | History | 20 mahasiswa
Biology 109 | Biology | 18 mahasiswa
Chemistry 105 | Chemistry | 28 mahasiswa

>> Hasil Projection <<

Math 101 | Mathematics | 15 mahasiswa
CS 102 | Computer Science | 25 mahasiswa
History 201 | History | 20 mahasiswa
Biology 109 | Biology | 18 mahasiswa
Chemistry 105 | Chemistry | 28 mahasiswa

>> Hasil setelah menambah field baru <<

Math 101 | Mathematics | 15 mahasiswa | kategori: low
CS 102 | Computer Science | 25 mahasiswa | kategori: high
History 201 | History | 20 mahasiswa | kategori: low
Biology 109 | Biology | 18 mahasiswa | kategori: low
Chemistry 105 | Chemistry | 28 mahasiswa | kategori: high


In [28]:
# HOMEWORK 
# 1 : Agregasi untuk menghitung jumlah course tiap departemen

# Menampilkan semua data 
print('📋 DATA :\n')
for course in courses_collection.find():
    print(f"{course['course']} | {course['department']} | {course['enrollments']} mahasiswa")

# Menggunakan agregasi
pipeline = [
    {'$group' : {'_id': '$department', 'jumlah_course': {'$sum': 1}}}
]

print("\n>> Jumlah course (matkul) per departemen <<\n")
for result in courses_collection.aggregate(pipeline):
    print(f"{result['_id']} | {result['jumlah_course']} mata kuliah")

📋 DATA :

Math 101 | Mathematics | 15 mahasiswa
CS 102 | Computer Science | 25 mahasiswa
History 201 | History | 20 mahasiswa
Biology 109 | Biology | 18 mahasiswa
Chemistry 105 | Chemistry | 28 mahasiswa

>> Jumlah course (matkul) per departemen <<

Mathematics | 1 mata kuliah
Computer Science | 1 mata kuliah
Biology | 1 mata kuliah
Chemistry | 1 mata kuliah
History | 1 mata kuliah


In [29]:
# 2 : $match dan $group untuk fmenampilkan matkul Computer Scirnce dengan enrollments > 25

# Menampilkan semua data 
print('📋 DATA :\n')
for course in courses_collection.find():
    print(f"{course['course']} | {course['department']} | {course['enrollments']} mahasiswa")

# Kombinasi $match dan $group
pipeline = [
    {'$match': {'department': 'Computer Science', 'enrollments': {'$gt': 25}}}, 
    {'$group': {'_id': '$department', 'total_course': {'$sum': 1}}}
]

print("\n>> Mata kuliah di departemen Computer Science dengan >25 mahasiswa <<\n")
for result in courses_collection.aggregate(pipeline):
    print(f"{result['_id']} | {result['total_course']} mata kuliah")

📋 DATA :

Math 101 | Mathematics | 15 mahasiswa
CS 102 | Computer Science | 25 mahasiswa
History 201 | History | 20 mahasiswa
Biology 109 | Biology | 18 mahasiswa
Chemistry 105 | Chemistry | 28 mahasiswa

>> Mata kuliah di departemen Computer Science dengan >25 mahasiswa <<



In [42]:
# 3 : $lookup untuk menggabungkan koleksi courses dengan students
# Buat data students menggunakan bulk_write
from pymongo import MongoClient, InsertOne

# Koneksi ke client MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['university_db']
students_collection = db['students']

# Supaya data tidak double saat dijalankan ulang
students_collection.delete_many({}) 

# Data students
students_operations = [
    InsertOne({'student_id': 1, 'name': 'Najeya', 'enrolled_course': 'Math 101'}),
    InsertOne({'student_id': 2, 'name': 'Reyya', 'enrolled_course': 'CS 102'}),
    InsertOne({'student_id': 3, 'name': 'Lenara', 'enrolled_course': 'CS 102'}),
    InsertOne({'student_id': 4, 'name': 'Kala', 'enrolled_course': 'History 201'}),
    InsertOne({'student_id': 5, 'name': 'Vergy', 'enrolled_course': 'Chemistry 105'}),
    InsertOne({'student_id': 6, 'name': 'Nalava', 'enrolled_course': 'Biology 109'}),
    InsertOne({'student_id': 7, 'name': 'Bae', 'enrolled_course': 'Math 101'}),
    InsertOne({'student_id': 8, 'name': 'Jena', 'enrolled_course': 'Chemistry 105'}),
    InsertOne({'student_id': 9, 'name': 'Daniel', 'enrolled_course': 'CS 102'}),
]
students_collection.bulk_write(students_operations)

print('>> Data students berhasil dimasukkan <<\n')
print('📋 DATA AWAL :\n')

# Menampilkan semua data di koleksi courses
for student in students_collection.find():
    print(f"{student['student_id']} | {student['name']} | {student['enrolled_course']}")

# Join courses dan students
#Pipeline agregasi untuk join kedua koleksi
pipeline = [
    {'$lookup': {'from': 'students', 'localField': 'course', 'foreignField': 'enrolled_course', 'as': 'student_details'}}
]

print("\n>> Hasil JOIN antara Courses dan Students <<\n")
for result in courses_collection.aggregate(pipeline):
    print(f"{result['course']} | {result['department']}")
    
    # Menampilkan nama mahasiswa 
    for name in result['student_details']:
        print(f"  - {name['name']}")
    print()


>> Data students berhasil dimasukkan <<

📋 DATA AWAL :

1 | Najeya | Math 101
2 | Reyya | CS 102
3 | Lenara | CS 102
4 | Kala | History 201
5 | Vergy | Chemistry 105
6 | Nalava | Biology 109
7 | Bae | Math 101
8 | Jena | Chemistry 105
9 | Daniel | CS 102

>> Hasil JOIN antara Courses dan Students <<

Math 101 | Mathematics
  - Najeya
  - Bae

CS 102 | Computer Science
  - Reyya
  - Lenara
  - Daniel

History 201 | History
  - Kala

Biology 109 | Biology
  - Nalava

Chemistry 105 | Chemistry
  - Vergy
  - Jena

